In [1]:
import pandas as pd
import datetime as dt
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from war_functions.pecota_tables import *
from cluster_luck_functions.cluster_luck_hitting import *
from cluster_luck_functions.cluster_luck_pitching import *
from cluster_luck_functions.cluster_luck_combined import *
from daily_adjustments.active_rosters import *
from daily_adjustments.BP_WAR import *
from daily_adjustments.todays_game_info import *
from daily_adjustments.starting_rotations_WAR import *
from daily_adjustments.adjusted_war_today import *
from odds_and_other_projections import *

In [2]:
# Gathering BOY Projections
boy_projections = pd.read_csv('data/preseason_projections.csv', index_col = 0)[['Team', 'Win_Percentage']]
boy_projections['Proj_Wins'] = boy_projections.Win_Percentage*162.0

In [3]:
# Saving preseason O/U and actual wins

preseason_ou = {
    'Diamondbacks' : 67.5,
    'Braves' : 91.5,
    'Cubs' : 75.5,
    'Reds' : 74.5,
    'Rockies' : 68.5,
    'Dodgers' : 98.5,
    'Marlins' : 77.5,
    'Brewers' : 89.5,
    'Mets' : 88.5,
    'Phillies' : 86.5,
    'Pirates' : 65.5,
    'Padres' : 89.5,
    'Giants' : 86.5,
    'Cardinals' : 84.5,
    'Nationals' : 71.5,
    'Orioles' : 62.5,
    'Red Sox' : 85.5,
    'White Sox' : 91.5,
    'Guardians' : 76.5,
    'Tigers' : 78.5,
    'Astros' : 91.5,
    'Royals' : 74.5,
    'Angels' : 83.5,
    'Twins' : 81.5,
    'Yankees' : 91.5,
    'Athletics' : 68.5,
    'Mariners' : 83.5,
    'Rays' : 89.5,
    'Rangers' : 74.5,
    'Blue Jays' : 92.5
}
wins = {
    'Diamondbacks' : 74,
    'Braves' : 101,
    'Cubs' : 74,
    'Reds' : 62,
    'Rockies' : 68,
    'Dodgers' : 111,
    'Marlins' : 69,
    'Brewers' : 86,
    'Mets' : 101,
    'Phillies' : 87,
    'Pirates' : 62,
    'Padres' : 89,
    'Giants' : 81,
    'Cardinals' : 93,
    'Nationals' : 55,
    'Orioles' : 83,
    'Red Sox' : 78,
    'White Sox' : 81,
    'Guardians' : 92,
    'Tigers' : 66,
    'Astros' : 106,
    'Royals' : 65,
    'Angels' : 73,
    'Twins' : 78,
    'Yankees' : 99,
    'Athletics' : 60,
    'Mariners' : 90,
    'Rays' : 86,
    'Rangers' : 68,
    'Blue Jays' : 92
}

In [4]:
# Filling DF with data
boy_projections['Preseason_OU'] = 0
for key, value in preseason_ou.items():
    boy_projections.loc[boy_projections.Team == key, 'Preseason_OU'] = value
    
boy_projections['Wins'] = 0
for key, value in wins.items():
    boy_projections.loc[boy_projections.Team == key, 'Wins'] = value

In [5]:
# Iterating through to generate bets and results

boy_projections['Bet_Hit'] = boy_projections.apply(lambda x: 1 if (x.Proj_Wins < x.Preseason_OU)
                                                   & (x.Wins < x.Preseason_OU) else 
                                                  1 if (x.Proj_Wins > x.Preseason_OU)
                                                  & (x.Wins > x.Preseason_OU) else -1, axis = 1)
boy_projections.columns

Index(['Team', 'Win_Percentage', 'Proj_Wins', 'Preseason_OU', 'Wins',
       'Bet_Hit'],
      dtype='object')

In [6]:
# Calling in game by game results to get final capital

results = pd.read_csv('results_tracker/results_tracker_base.csv', index_col = 0)
capital = results.loc[len(results)-1, 'Money_Tracker']

In [7]:
# Getting final capital

final_capital = capital + 2000 * sum(boy_projections.Bet_Hit)
final_capital

130825.83388360712